In [5]:
pip install dash plotly pandas jupyter-dash

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import os
import base64  # For base64 encoding


app = Dash(__name__, suppress_callback_exceptions=True)

df = pd.read_csv('energy_analyzed final.csv')

country_list = ['India', 'Australia', 'United States', 'France', 'New Zealand']


global_policy_data = {
    "Year": [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    "India": [6, 7, 8, 8, 9, 11, 11, 13, 13, 15, 17, 17, 19, 20],
    "France": [5, 5, 7, 7, 8, 8, 10, 10, 10, 12, 13, 14, 16, 17],
    "United States": [7, 8, 8, 9, 10, 11, 13, 15, 17, 17, 19, 20, 22, 23],
    "New Zealand": [3, 3, 3, 4, 4, 6, 6, 7, 7, 9, 9, 10, 9, 11],
    "Australia": [5, 5, 6, 6, 7, 8, 8, 9, 9, 11, 11, 11, 11, 11]
}


df_pivot = df.pivot_table(index=['Country', 'Energy'], columns='Sentiment_Mapped', aggfunc='size', fill_value=0)
df_pivot_percentage = df_pivot.div(df_pivot.sum(axis=1), axis=0) * 100



df_pivot_percentage = df_pivot.div(df_pivot.sum(axis=1), axis=0) * 100


file_path = 'Public Opnion Accuracy.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')


grouped_data = data.groupby(['Country', 'Public_Opinion_Accuracy']).size().reset_index(name='Count')


pastel_colors = {
    'Australia': ['#ffb3ba', '#bae1ff'],  # Soft Red, Soft Blue
    'France': ['#baffc9', '#ffdfba'],     # Soft Green, Soft Orange
    'India': ['#ffd1a1', '#b3e5fc'],      # Soft Peach, Soft Sky Blue
    'United States Of America': ['#ffdfd3', '#d3ffce'],        # Coral, Mint Green
    'New Zealand': ['#ffb347', '#c2f0c2'] # Peach, Pale Green
}


app.layout = html.Div([
    html.H1("Public Opinion Accuracy by Country"),

    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in grouped_data['Country'].unique()],
        value='Australia'  # Default value
    ),

    dcc.Graph(id='donut-chart')
])


def create_line_chart(data, title):
    fig = go.Figure(data=[
        go.Scatter(x=data['Year'], y=data[country], mode='lines+markers', name=country)
        for country in data if country != 'Year'
    ])
    fig.update_layout(title=title, xaxis_title='Year', yaxis_title='Policies')
    return fig


def plot_renewable_energy_trends(file_paths):
    fig = go.Figure()
    for path in file_paths:
        data = pd.read_csv(path)
        avg = data.groupby('Year')['renewable_energy'].mean().reset_index()
        country = path.split('_')[0].capitalize()
        fig.add_trace(go.Scatter(x=avg['Year'], y=avg['renewable_energy'], mode='lines', name=country))
    fig.update_layout(
        title='Renewable Energy Search Trends (2004-2024)',
        xaxis_title='Year', yaxis_title='Popularity'
    )
    return fig


challenges_per_country = df.groupby(['Country', 'reasons']).size().unstack(fill_value=0)
challenges_percent = challenges_per_country.div(challenges_per_country.sum(axis=1), axis=0) * 100

challenges_per_energy = df.groupby('Energy')['reasons'].value_counts().unstack(fill_value=0)
challenges_percent_energy = challenges_per_energy.div(challenges_per_energy.sum(axis=1), axis=0) * 100


country_image_mapping = {
    'India': 'India.jpg',
    'Australia': 'Australia.jpg',
    'United States': 'USA.jpg',  # Mapping for United States -> USA
    'France': 'France.jpg',
    'New Zealand': 'New Zealand.jpg'
}


def encode_image(image_path):
    with open(image_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('ascii')
    return f"data:image/jpg;base64,{encoded_image}"


app.layout = html.Div([
    html.H1('Indo-Pacific Renewable Energy Dashboard',
            style={'textAlign': 'center', 'padding': '20px', 'background-color': '#1abc9c', 'color': 'white'}),

    # First row: Line charts
    html.Div([
        dcc.Graph(
            id='global-policy-graph',
            figure=create_line_chart(global_policy_data, 'Global Policy Trends')
        ),
        dcc.Graph(
            id='renewable-trends',
            figure=plot_renewable_energy_trends([
                'Australia_trends_processed.csv', 'France_trends_processed.csv',
                'India_trends_processed.csv', 'New zealand_trends_processed.csv',
                'USA_trends_processed.csv'
            ])
        )
    ], style={'display': 'flex', 'justify-content': 'space-between', 'margin-bottom': '20px'}),

    
    html.Div([
        dcc.Slider(
            id='country-slider', min=0, max=len(df['Country'].unique()) - 1, step=1,
            marks={i: country for i, country in enumerate(df['Country'].unique())}, value=0
        ),
        dcc.Graph(id='heatmap')
    ], style={'margin-bottom': '20px'}),

    
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='country-dropdown', options=[{'label': c, 'value': c} for c in df['Country'].unique()],
                value='Australia', style={'width': '80%', 'margin': 'auto'}
            ),
            dcc.Graph(id='reason-distribution-graph')
        ], style={'width': '48%'}),

        html.Div([
            dcc.Dropdown(
                id='chart-selector',
                options=[
                    {'label': 'Challenges by Country', 'value': 'country'},
                    {'label': 'Challenges by Energy Type', 'value': 'energy'}
                ],
                value='country', style={'width': '80%', 'margin': 'auto'}
            ),
            dcc.Graph(id='challenge-overview-chart')
        ], style={'width': '48%'})
    ], style={'display': 'flex', 'justify-content': 'space-between', 'margin-bottom': '20px'}),

    
    html.H2("Public Opinion Accuracy by Country"),
    dcc.Dropdown(
        id='opinion-country-dropdown',  # Changed ID here
        options=[{'label': country, 'value': country} for country in grouped_data['Country'].unique()],
        value='Australia'
    ),
    dcc.Graph(id='donut-chart'),

    
    html.Div([
        dcc.Graph(id='world-map', figure=px.choropleth(
            locations=country_list, locationmode='country names',
            projection='equirectangular', title='Countries Working on Renewable Energy'
        ))
    ]),

    
    html.Div(id='map-or-image')
], style={'padding': '20px', 'background-color': '#f8f9fa'})



@app.callback(
    Output('heatmap', 'figure'),
    Input('country-slider', 'value')
)
def update_heatmap(selected_country_index):
    # Get the selected country
    country = df['Country'].unique()[selected_country_index]
    # Get the data for the selected country
    country_data = df_pivot_percentage.loc[country]

    
    fig = px.imshow(
        country_data,
        labels={'x': 'Energy Type', 'y': 'Sentiment', 'color': 'Percentage'},
        title=f'Sentiment Analysis for {country}',
        color_continuous_scale='Viridis'
    )

    
    fig.update_layout(
        xaxis_title='Energy Type',
        yaxis_title='Sentiment',
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False)
    )

    return fig

@app.callback(
    Output('reason-distribution-graph', 'figure'),
    Input('country-dropdown', 'value')
)
def update_reason_distribution(selected_country):
    data = challenges_percent.loc[selected_country]
    fig = px.bar(data, x=data.index, y=data.values, labels={'x': 'Reason', 'y': 'Percentage'},
                 title=f'Public Identified Challenges in {selected_country}')
    return fig

@app.callback(
    Output('challenge-overview-chart', 'figure'),
    Input('chart-selector', 'value')
)
def update_challenge_overview(selected_option):
    data = challenges_percent if selected_option == 'country' else challenges_percent_energy
    fig = px.bar(data, barmode='stack', title=f'Challenges by {selected_option.capitalize()}')
    return fig


@app.callback(
    Output('donut-chart', 'figure'),
    [Input('opinion-country-dropdown', 'value')]  # Changed Input ID here
)
def update_chart(selected_country):
    # Filter data for the selected country
    country_data = grouped_data[grouped_data['Country'] == selected_country]

    # Create the donut chart using Plotly
    fig = px.pie(country_data, values='Count', names='Public_Opinion_Accuracy',
                 title=f'Public Opinion Accuracy for {selected_country}',
                 hole=0.4, color_discrete_sequence=pastel_colors.get(selected_country, px.colors.qualitative.Pastel))

    return fig

@app.callback(
    Output('map-or-image', 'children'),
    Input('world-map', 'clickData')
)

def display_image_or_map(clickData):
    if clickData:
        country = clickData['points'][0]['location']
        # Use the mapping to get the corresponding image file name
        image_file = country_image_mapping.get(country, None)  # None if country not mapped
        if image_file:
            # image_path = os.path.join('/assets', image_file)  # Path in Colab's /content directory
            image_path = os.path.join('assets', f'{country}.jpg')
            if os.path.exists(image_path):  # Check if the image exists in /content
                encoded_image = encode_image(image_path)  # Encode the image to base64
                # return html.Img(src=encoded_image, style={'max-width': '80%', 'margin': 'auto'})
                return html.Img(src=f'/assets/{country}.jpg', style={'max-width': '80%', 'margin': 'auto'})
        return html.H3(f'No image available for {country}', style={'text-align': 'center'})
    return html.H3('Click on a country to view its image', style={'text-align': 'center'})

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=9020)